'''<br>
@Author: Rahul <br>
@Date: 2024-10-24<br>
@Last Modified by: Rahul <br>
@Last Modified time: 2024-10-24<br>
@Title: Python program to do CRUD Operation in DynamoDB<br>
'''

In [1]:
import boto3
from botocore.exceptions import ClientError
from pprint import pprint
from decimal import Decimal
import time

Create Table

In [2]:

# Create a DynamoDB client
client = boto3.client('dynamodb')

# Create DynamoDB table
def create_movie_table():
    table = client.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table

movie_table = create_movie_table()
print("Create DynamoDB table succeeded............")
print("Table status: {}".format(movie_table['TableDescription']['TableStatus']))


Insert Into Table

In [9]:

def put_movie(title, year, plot, rating):
    response = client.put_item(
       TableName='Movies',
       Item={
            'year': {
                'N': "{}".format(year),
            },
            'title': {
                'S': "{}".format(title),
            },
            'plot': {
                "S": "{}".format(plot),
            },
            'rating': {
                "N": "{}".format(rating),
            }
        }
    )
    return response

response = put_movie("The Big New Movie", 2015, "A movie about big things", 8.5)
print("Movie inserted into DynamoDB:", response)


Movie inserted into DynamoDB: {'ResponseMetadata': {'RequestId': '9033B526JB4QQ8VNR1LPJHV4RJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Thu, 24 Oct 2024 12:45:26 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '9033B526JB4QQ8VNR1LPJHV4RJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


Get Movie

In [6]:

# Create a DynamoDB client
client = boto3.client('dynamodb')

def get_movie(title, year):
    try:
        response = client.get_item(       
            TableName='Movies',
            Key={
                'year': {
                    'N': "{}".format(year),
                },
                'title': {
                    'S': "{}".format(title),
                }
            }
        )
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        return response.get('Item')

movie = get_movie("The Big New Movie", 2015)

if movie:
    print("Get an item from DynamoDB succeeded............")
    pprint(movie, sort_dicts=False)
else:
    print("Movie not found in DynamoDB.")


Get an item from DynamoDB succeeded............
{'year': {'N': '2015'},
 'plot': {'S': 'A movie about big things'},
 'rating': {'N': '8.5'},
 'title': {'S': 'The Big New Movie'}}


Update Table

In [7]:
def update_movie(title, year, rating, plot, actors):
    response = client.update_item(
        TableName='Movies',
        Key={
            'year': {
                    'N': "{}".format(year),
            },
            'title': {
                    'S': "{}".format(title),
            }
        },
        ExpressionAttributeNames={
            '#R': 'rating',
            '#P': 'plot',
            '#A': 'actors'
        },
        ExpressionAttributeValues={
            ':r': {
                'N': "{}".format(rating),
            },
            ':p': {
                'S': "{}".format(plot),
            },
            ':a': {
                'SS': actors,
            }
        },
        UpdateExpression='SET #R = :r, #P = :p, #A = :a',
        ReturnValues="UPDATED_NEW"
    )
    return response

update_response = update_movie( "The Big New Movie", 2015, 5.5, "Everything happens all at once.",["Larry", "Moe", "Curly"])
print("Update and item in  DynamoDB succeeded............")
pprint(update_response, sort_dicts=False)

Update and item in  DynamoDB succeeded............
{'Attributes': {'plot': {'S': 'Everything happens all at once.'},
                'rating': {'N': '5.5'},
                'actors': {'SS': ['Curly', 'Larry', 'Moe']}},
 'ResponseMetadata': {'RequestId': 'B2P5S0JTMPQH7J14BGINF6HQAVVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'HTTPStatusCode': 200,
                      'HTTPHeaders': {'server': 'Server',
                                      'date': 'Thu, 24 Oct 2024 12:28:50 GMT',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'content-length': '124',
                                      'connection': 'keep-alive',
                                      'x-amzn-requestid': 'B2P5S0JTMPQH7J14BGINF6HQAVVV4KQNSO5AEMVJF66Q9ASUAAJG',
                                      'x-amz-crc32': '2033653244'},
                      'RetryAttempts': 0}}


Delete Operation

In [8]:
def delete_underrated_movie(title, year, rating):
    try:
        response = client.delete_item(
            TableName='Movies',
            Key={
                'year': {
                    'N': "{}".format(year),
                },
                'title': {
                    'S': "{}".format(title),
                }
            },
            ConditionExpression="rating <= :a",
            ExpressionAttributeValues={
                ':a': {
                    'N': "{}".format(rating),
                }
            }
        )
    except ClientError as e:
        if e.response['Error']['Code'] == "ConditionalCheckFailedException":
            print(e.response['Error']['Message'])
        else:
            raise
    else:
        return response 

delete_response = delete_underrated_movie("The Big New Movie", 2015, 7.5)
if delete_response:
    print("Delete an Item in DynamoDB table.........................")
    pprint(delete_response, sort_dicts=False)

Delete an Item in DynamoDB table.........................
{'ResponseMetadata': {'RequestId': 'EM85RPCEC0VETPAUTC3I6U8OJFVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'HTTPStatusCode': 200,
                      'HTTPHeaders': {'server': 'Server',
                                      'date': 'Thu, 24 Oct 2024 12:30:13 GMT',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'content-length': '2',
                                      'connection': 'keep-alive',
                                      'x-amzn-requestid': 'EM85RPCEC0VETPAUTC3I6U8OJFVV4KQNSO5AEMVJF66Q9ASUAAJG',
                                      'x-amz-crc32': '2745614147'},
                      'RetryAttempts': 0}}
